# Base de Datos II - Proyecto II: Indice invertido

## Integrantes
- Nicho, Jorge
- Riveros, Andres
- Tanta, Johan

In [108]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/johan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/johan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [109]:
import os
import json
import sys
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import re
import math 
from decimal import Decimal
import regex
import numpy as np
from collections import Counter

In [110]:
def stopwords_stemmer(tweet_text):
    tokens = nltk.word_tokenize(tweet_text)
    stoplist = stopwords.words("spanish")
    stoplist += ['/…','--','RT','`','@','|','¿','?', '¡', '!','aqui', '.', ',', ';', '«', '»', ':', '(', ')', '"','en', 'En', 'la', 'La', 'con', 'Con','sin', 'Sin', 'al', 'Al', 'de', 'De', 'el', 'El', '#', '$', '^', '&', '*', '%']
    tokens_clean = tokens.copy()

    for token in tokens:
        if token in stoplist:
            tokens_clean.remove(token)
            
    stemmer = SnowballStemmer("spanish")
    for i in range(len(tokens_clean)):
        tokens_clean[i] = stemmer.stem(tokens_clean[i])
        
    return tokens_clean

In [111]:
json_path = './data/data_elecciones/'
tweets_folder = os.listdir(json_path)
inverted_index = {}
term_termid_dict = {}
filenameid_filename_dict = {}
tweet_termids_dict = {}
N = 0
NTerms = 0
for tweets_file in tweets_folder:
    filenameid = len(filenameid_filename_dict)
    filenameid_filename_dict[filenameid] = tweets_file
    with open(json_path + tweets_file, encoding="utf8") as file:
        json_content = json.load(file)
        for tweet in json_content:
            N += 1
            tokens_clean = stopwords_stemmer(tweet["text"])
            each_term_frequency = Counter(tokens_clean)
            tweet_id = tweet["id"]
            norma = 0
            tweet_termids_dict[tweet_id] = [filenameid, {}, norma]
            for term in each_term_frequency:
                freq = each_term_frequency[term]
                if not (term in term_termid_dict):
                    term_termid_dict[term] = len(term_termid_dict)
                
                term_id = term_termid_dict[term]
                tweet_termids_dict[tweet_id][1][term_id] = 0.0

                if not (term_id in inverted_index):
                    inverted_index[term_id] = {tweet_id : freq}
                else:
                    inverted_index[term_id][tweet_id] = freq
    print(tweets_file)

termid_df_dict = {}
for term_id in inverted_index:
    posting_list = inverted_index[term_id]
    termid_df_dict[term_id] = len(posting_list)
NTerms = len(term_termid_dict)

tweets_2018-09-28.json
tweets_2018-10-04.json
tweets_2018-09-06.json
tweets_2018-09-16.json
tweets_2018-08-31.json
tweets_2018-08-24.json
tweets_2018-08-13.json
tweets_2018-10-01.json
tweets_2018-09-09.json
tweets_2018-08-12.json
tweets_2018-09-11.json
tweets_2018-09-08.json
tweets_2018-09-29.json
tweets_2018-09-10.json
tweets_2018-09-07.json
tweets_2018-10-03.json
tweets_2018-08-30.json
tweets_2018-09-19.json
tweets_2018-08-19.json
tweets_2018-09-04.json
tweets_2018-09-15.json
tweets_2018-09-24.json
tweets_2018-10-02.json
tweets_2018-09-17.json
tweets_2018-08-25.json
tweets_2018-08-09.json
tweets_2018-08-20.json
tweets_2018-09-25.json
tweets_2018-08-21.json
tweets_2018-08-18.json
tweets_2018-09-27.json
tweets_2018-10-05.json
tweets_2018-08-16.json
tweets_2018-09-14.json
tweets_2018-09-03.json
tweets_2018-08-26.json
tweets_2018-08-07.json
tweets_2018-09-21.json
tweets_2018-08-22.json
tweets_2018-09-05.json
tweets_2018-08-14.json
tweets_2018-08-10.json
tweets_2018-09-12.json
tweets_2018

In [112]:
def getWeightTF_IDF(posting_list, tweet_id):
    tf = posting_list[tweet_id]
    df = float(len(posting_list))
    return math.log(1+tf, 10) * math.log(N / df, 10)

In [113]:
import numpy as np

def setWeightTF_IDF():
    for term_id in inverted_index:
        posting_list = inverted_index[term_id]
        for tweet_id in posting_list:
            tweet_termids_dict[tweet_id][1][term_id] = getWeightTF_IDF(posting_list, tweet_id)

def getNorma(posting_list):
    values = np.array(list(posting_list.values()))
    return np.linalg.norm(values)

def setNorma():
    for tweet_id in tweet_termids_dict:
        posting_list = tweet_termids_dict[tweet_id][1]
        tweet_termids_dict[tweet_id][2] = getNorma(posting_list)
    
setWeightTF_IDF()
setNorma()

In [114]:
import pickle
bin_path = './bin/'
term_termid_dict_file = open(bin_path + "term_termid_dict.dat", "wb")
inverted_index_file = open(bin_path + "inverted_index.dat", "wb")
tweet_termids_dict_file = open(bin_path + "tweet_termids_dict.dat", "wb")
filenameid_filename_dict_file = open(bin_path + "filenameid_filename_dict.dat", "wb")
termid_df_dict_file = open(bin_path + "termid_df_dict.dat", "wb")
N_file = open(bin_path + "N.dat", "wb")
NTerms_file = open(bin_path + "NTerms.dat", "wb")

pickle.dump(term_termid_dict, term_termid_dict_file)
pickle.dump(inverted_index, inverted_index_file)
pickle.dump(tweet_termids_dict, tweet_termids_dict_file)
pickle.dump(filenameid_filename_dict, filenameid_filename_dict_file)
pickle.dump(termid_df_dict, termid_df_dict_file)
pickle.dump(N, N_file)
pickle.dump(NTerms, NTerms_file)

term_termid_dict_file.close()
inverted_index_file.close()
tweet_termids_dict_file.close()
filenameid_filename_dict_file.close()
N_file.close()
NTerms_file.close()

In [116]:
print("Cantidad de terminos: ", NTerms)
print("Cantidad de Tweets: ", N)

Cantidad de terminos:  128562
Cantidad de Tweets:  479945
